In [1]:
import warnings
# from PIL import Image, ImageEnhance
warnings.filterwarnings('ignore')
# import tensorflow as tf
# from keras.models import load_model
# from keras.applications.vgg16 import preprocess_input
# import numpy as np
# from keras.preprocessing import image

# from PyQt5 import QtCore, QtGui, QtWidgets
# from PyQt5.QtWidgets import QFileDialog
# from PyQt5.QtGui import QMovie
# from PyQt5.QtWidgets import QMessageBox

# from win32com.client import Dispatch

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten

In [3]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE = [224, 224]

train_path = 'Dataset_kaggle/train'
test_path = 'Dataset_kaggle/test'

In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
for layer in vgg.layers:
    layer.trainable = False

In [7]:
folders = glob('Dataset_kaggle/train/*')
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)
# Create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model 
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Make sure that you provide same target size as initialized for the image size 
training_set = train_datagen.flow_from_directory('Dataset_kaggle/train',
                                                 target_size=(224, 224),
                                                 batch_size=10,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('Dataset_kaggle/test',
                                                 target_size=(224, 224),
                                                 batch_size=10,
                                                 class_mode='categorical')

Found 6326 images belonging to 4 classes.
Found 771 images belonging to 4 classes.


In [11]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=1,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

633/633 [==============================] - 1032s 2s/step - loss: 0.2990 - accuracy: 0.8985 - val_loss: 0.6895 - val_accuracy: 0.7925


In [12]:
img = tf.keras.utils.load_img('Dataset_kaggle/val/COVID19/COVID19(566).jpg', target_size=(224,224))

In [13]:
x=tf.keras.utils.img_to_array(img)

In [14]:
x=np.expand_dims(x, axis = 0)

In [15]:
img_data=preprocess_input(x)

In [16]:
classes=model.predict(img_data)

1/1 [==============================] - 0s 264ms/step


In [17]:
classes

array([[0., 0., 0., 1.]], dtype=float32)

In [18]:
result=classes[0][0]

In [19]:
train_datagen = tf.keras.preprocessing.image_dataset_from_directory("Dataset_kaggle/train", shuffle = True, image_size=(64, 64), batch_size=32)
test_datagen = tf.keras.preprocessing.image_dataset_from_directory("Dataset_kaggle/test", shuffle = True, image_size=(64, 64), batch_size=32)
class_names = train_datagen.class_names
print(class_names)

Found 6326 files belonging to 4 classes.
Found 771 files belonging to 4 classes.
['COVID19', 'NORMAL', 'PNEUMONIA', 'TURBERCULOSIS']


In [25]:
import numpy as np
img = tf.keras.utils.load_img('Dataset_kaggle/test/COVID19/COVID19(462).jpg', target_size=(224, 224))
x=tf.keras.utils.img_to_array(img)
x=np.array([x])

predi=model.predict(x)
print(np.argmax(predi[0]))
print(predi[0])
predicted_class = class_names[np.argmax(predi[0])] 
confidence = round(100 * (np.max(predi[0])), 2)
print(predicted_class)
print(confidence)

1/1 [==============================] - 0s 157ms/step
0
[9.9986720e-01 1.3279149e-04 2.3214069e-26 0.0000000e+00]
COVID19
99.99


In [26]:
model.save('xray_model.h5')

In [27]:
from keras.models import load_model
load_model('xray_model.h5')